In [12]:
import pandas as pd
import numpy as np

# Load the dataset and visualize it

df = pd.read_csv("train.tsv", sep="\t")

df.drop(columns=["2635.json", "dwayne-bohac", "State representative", "Texas", "republican", "a mailer"], inplace=True)
df.rename(columns={"": "Index",
                    "false": "correctness",
                    "Says the Annies List political group supports third-trimester abortions on demand.": "Text",
                    "abortion": "Theme",
                    "0": "barely true counts",
                    "1": "false counts",
                    "0.1": "half true counts",
                    "0.2": "mostly true counts",
                    "0.3": "pants on fire counts",}, inplace=True)

df

,correctness,Text,Theme,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts
0,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",0.0,0.0,1.0,1.0,0.0
1,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,70.0,71.0,160.0,163.0,9.0
2,false,Health care reform legislation is likely to ma...,health-care,7.0,19.0,3.0,5.0,44.0
3,half-true,The economic turnaround started at the end of ...,"economy,jobs",15.0,9.0,20.0,19.0,2.0
4,true,The Chicago Bears have had more starting quart...,education,0.0,3.0,2.0,5.0,1.0
...,...,...,...,...,...,...,...,...
10234,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",0.0,1.0,1.0,1.0,0.0
10235,mostly-true,Democrats have now become the party of the [At...,elections,0.0,0.0,0.0,1.0,0.0
10236,half-true,Says an alternative to Social Security that op...,"retirement,social-security",4.0,11.0,5.0,3.0,3.0
10237,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",3.0,1.0,3.0,0.0,0.0
